# Development Training fit

Model research script. This is a testing ground for the production functions.

In [13]:
#Import libraries
from data_proc import *
import datetime
import pandas as pd
import numpy as np

In [14]:
#Import Keras module
import tensorflow as tf

In [15]:
#Establish Cloud SQL connection
import MySQLdb
conn = MySQLdb.connect(host="35.194.155.252", user="root", passwd="60761271", db="HSI_DATA")

In [16]:
#Check GPU
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [17]:
#pretty plots
%matplotlib inline

## Fetch Data

In [18]:
#Define forward return step size
step = 1
upper_cutoff = 0.0005
lower_cutoff = -0.0005

In [19]:
#Define Training, Validation, test date
training_start = datetime.date(2007, 8, 31)
training_end = datetime.date(2015, 12, 30)
validation_start = datetime.date(2015, 12, 31)
validation_end = datetime.date(2016, 5, 31)
test_start = datetime.date(2017, 6, 1)
latest_close = datetime.date.today() + datetime.timedelta(days=-3)

In [20]:
latest_close

datetime.date(2017, 12, 7)

In [21]:
#Fetch training data
X = pd.concat([get_data(conn, training_start, latest_close, 'GLOBAL_INDICES', '*'), 
                      get_data(conn, training_start, latest_close, 'hsi_data', '*')], axis=1)

In [22]:
#Fetch forward returns
y = get_fwd_ret(conn, training_start, latest_close, step)
y = get_labels_from_fwd_ret(y, upper_cutoff=upper_cutoff, lower_cutoff=lower_cutoff)

In [23]:
#Close connection
conn.close()

## Preprocess the input and labels

In [24]:
#Split data into train, validation and test
_, X_train, X_val, X_test = train_val_test_split(X, training_start, validation_start, test_start, latest_close, 'RobustScaler')
_, y_train, y_val, y_test = train_val_test_split(y, training_start, validation_start, test_start, latest_close, None)

### Fit LSTM Model

Fit LSTM model based on previous research

In [25]:
def build_inputs(batch_size, num_steps):
    
    #===========================================================================================
    #Function to build tensor inputs to tensorflow
    #---------------------------------------------
    #batch_size: Number of samples per batch
    #num_steps: Number of sequences per sample
    #===========================================================================================
    
    #Declare the placeholders
    inputs = tf.placeholder(tf.float32, shape=(batch_size, num_steps, 74), name='inputs')
    targets = tf.placeholder(tf.int32, shape=(batch_size, num_steps, 3), name='labels')
    
    #Dropout layer probabilities
    keep_prob = tf.placeholder(tf.float32 , name='keep_prob')
        
    return inputs, targets, keep_prob

In [26]:
def build_network(lstm_size, num_layers, batch_size, keep_prob):
    
    #===========================================================================================
    #Function to build network. The network will be the following
    #LSTM > Dropout > LSTM > Dropout > Dense > Softmax
    #---------------------------------------------
    #lstm_size: Number of samples per batch
    #num_layers: Number of layers for LSTM
    #batch_size: Number of sequences per batch
    #keep_prob: Dropout probability for the 2 dropout layers
    #dense_hidden_units: Number of hidden units within the dense layer
    #===========================================================================================
    
    def build_cell(size, keep_prob):
        
        #Helper function to create an LSTM cell, just to make things a bit neat
        lstm = tf.contrib.rnn.LSTMCell(size)
        
        #Add drop out
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return drop
    
    #Stack up LSTM layers, and then add a dense layer
    cells = [build_cell(lstm_size, keep_prob) for _ in range(0, num_layers)]
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(cells)
        
    #Initial State
    initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
    
    return stacked_lstm, initial_state

In [27]:
def build_output(stacked_lstm, dense_hidden_units, in_size, n_class=3):
    
    #===========================================================================================
    # Function to build output function
    #---------------------------------------------
    # dense: input tensor from dense layer
    # n_class: number of class probabilities
    #===========================================================================================
    
    #We have the dense layer, build logits
    seq_output = tf.concat(stacked_lstm, axis=1)
    print('seq_output:', seq_output.get_shape())
    
    x = tf.reshape(seq_output, [-1, in_size])
    print('x:', x.get_shape())
    
    dense = tf.layers.dense(inputs=x, units=dense_hidden_units, activation=tf.nn.relu)
    
    softmax_w = tf.Variable(tf.truncated_normal((dense_hidden_units, n_class), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_class))
    
    #Calculate logit
    logits = tf.matmul(dense, softmax_w) + softmax_b
    print('logits:', logits.get_shape())
    #Output
    out = tf.nn.softmax(logits, name='predictions')
    print('out:', out.get_shape())
    
    return out, logits
    

In [28]:
def build_loss(logits, targets, n_class=3):
    
    #===========================================================================================
    # Function to build output function
    #---------------------------------------------
    # dense: input tensor from dense layer
    # n_class: number of class probabilities
    #===========================================================================================
    
    print('build_loss logits:', logits.get_shape())
    print('build_loss targets:', targets.get_shape())
    
    #One-hot encode targets
    y_one_hot = tf.one_hot(targets, n_class)
    print('y_one_hot:', y_one_hot.get_shape())
    y_reshaped =  tf.reshape(y_one_hot, logits.get_shape())
    print('y_reshaped:', y_reshaped.get_shape())
    
    #Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss
    

In [29]:
def build_optimizer(loss, learning_rate, grad_clip):
    
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, vars))
    
    return optimizer

In [30]:
#Define LSTM Parameters and structure
class DirectionLSTM:
    
    #============================================================================================
    #Class Constructor
    #-----------------
    #Construct LSTM object to carry around
    #self.params: dictionary of model parameters
    #self.model: Keras functional API model storing the network architect
    #============================================================================================
    def __init__(self, n_class=3, batch_size=128, num_steps=10, n_hidden_units=(100, 100, 50), num_layers=2,
                 learning_rate=0.001, grad_clip=2, online=False):
        
        #If Online learning, we will be having batch_size of 1 and 1 time step
        if online == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps
        
        #Reset graph
        tf.reset_default_graph()
        
        #Build inputs
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)
        
        #Build LSTM Cell
        cell, self.initial_state = build_network(93, num_layers, batch_size, self.keep_prob)
        
        #Link Up the RNN cells
        outputs, state = tf.nn.dynamic_rnn(cell, self.inputs, initial_state=self.initial_state)
        self.final_state = state
        
        #Get softmax predictions
        self.prediction, self.logits = build_output(outputs, 66, 93, n_class)
        
        #Loss and optimizer
        self.loss = build_loss(self.logits, self.targets, n_class)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)
    

In [31]:
#Model Parameters
#DROP_OUT = (0.26036828997037875, 0.10581996841419966)
DROP_OUT = 0.5
LEARN_RATE = 0.0002572041248224521
N_HIDDEN = (93, 89, 66)
N_FEATURES = X.shape[1]
TIMESTEP = 11
BATCH_SIZE = 128

In [32]:
epochs = 20
# Save every N iterations
#save_every_n = 200

model = DirectionLSTM(n_class=3, batch_size=BATCH_SIZE, num_steps=TIMESTEP,
                n_hidden_units=N_HIDDEN, num_layers=2, 
                learning_rate=LEARN_RATE)

#saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batch(X_train, y_train, BATCH_SIZE, TIMESTEP):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: DROP_OUT,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            #if (counter % save_every_n == 0):
                #saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    #saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

seq_output: (128, 11, 93)
x: (1408, 93)
logits: (1408, 3)
out: (1408, 3)
build_loss logits: (1408, 3)
build_loss targets: (128, 11, 3)
y_one_hot: (128, 11, 3, 3)


ValueError: Cannot reshape a tensor with 12672 elements to shape [1408,3] (4224 elements) for 'Reshape_1' (op: 'Reshape') with input shapes: [128,11,3,3], [2] and with input tensors computed as partial shapes: input[1] = [1408,3].

In [ ]:
for (x,y) in get_batch(X_train, y_train, BATCH_SIZE, TIMESTEP):
    print(x.shape, y.shape)
    

In [ ]:
zip((1,2,3), (1,2,3))

In [ ]:
for (x,y) in zip((1,2,3), (1,2,3)):
    print(x,y)